In [6]:
import numpy as np
import pandas as pd
import re
import nltk

In [7]:
pitch_file = 'shark_tank_pitch_detail.csv'

load2 = pd.read_csv(pitch_file)
load2.head()

,Season_Epi_code,Pitched_Business_Identifier,Pitched_Business_Desc,Deal_Status,Deal_Shark
0,101,Ava the Elephant,"(Emmy the Elephant during show, trademarked a...",1,BC
1,101,College Foxes Packing Boxes,a packing and organizing service based on an a...,0,NaN
2,101,Ionic Ear,an implantable Bluetooth device requiring surg...,0,NaN
3,101,Mr. Tod's Pie Factory,a pie company,1,BC+DJ
4,101,Wispots,an electronic hand-held device for waiting roo...,0,NaN


In [8]:
def data_cleaning(corpus):
    letters_only = re.sub("[^a-zA-Z]", " ", corpus) 
    words = letters_only.lower().split()                            
    return( " ".join( words ))     

In [9]:
from nltk.corpus import stopwords

In [10]:
addedwords = ('use','line','allow','make','offer','provide','products','design','made','similar','like','play','creates','created','unique')

stop = stopwords.words('english')+list(addedwords)

load2['Pitched_Business_Desc'] = load2['Pitched_Business_Desc'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\deler/nltk_data'
    - 'C:\\Users\\deler\\anaconda3\\nltk_data'
    - 'C:\\Users\\deler\\anaconda3\\share\\nltk_data'
    - 'C:\\Users\\deler\\anaconda3\\lib\\nltk_data'
    - 'C:\\Users\\deler\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************


In [11]:
load2['Pitched_Business_Desc'] = load2['Pitched_Business_Desc'].apply(lambda x:data_cleaning(x))
load2 = load2[['Deal_Status','Pitched_Business_Desc']]
for i in range(5):
    print(load2['Pitched_Business_Desc'][i])

emmy the elephant during show trademarked as ava after a plastic elephant shaped device that helps parents give small children oral medicine emmy the elephant during show trademarked as ava after a plastic elephant shaped device that lets parents easily give small children oral medicine
a packing and organizing service based on an already successful business called college hunks hauling junk a spinoff professional packing and organizing service based on an already successful junk removal and moving business called college hunks hauling junk
an implantable bluetooth device requiring surgery to insert the device into the user s head an implantable bluetooth device requiring surgery into your head
a pie company
an electronic hand held device for waiting rooms an electronic hand held device for waiting rooms


In [12]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(load2,test_size=0.2)

In [13]:
train_corpus = []
test_corpus = []
for each in train['Pitched_Business_Desc']:
    train_corpus.append(each)
for each in test['Pitched_Business_Desc']:
    test_corpus.append(each)
## Start creating them
from sklearn.feature_extraction.text import CountVectorizer
v = CountVectorizer(ngram_range=(2,2))
train_features = v.fit_transform(train_corpus)
test_features=v.transform(test_corpus)

In [14]:
print(train_features.shape)
print(test_features.shape)

(564, 11340)
(142, 11340)


In [15]:
# Import ML models from sklearn
from sklearn.linear_model import LogisticRegression # Regression classifier
from sklearn.tree import DecisionTreeClassifier # Decision Tree classifier
from sklearn import svm # Support Vector Machine
from sklearn.linear_model import SGDClassifier # Stochastic Gradient Descent Classifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier # Random Forest and Gradient Boosting Classifier
from sklearn.naive_bayes import MultinomialNB # Naive Bayes Classifier 
from sklearn.metrics import accuracy_score, recall_score, confusion_matrix # Some metrics to check the performance of the models

In [16]:
#these are tunable to achieve max accuracy


Classifiers = {'LR':LogisticRegression(random_state=10,C=5,max_iter=200),
               'DTC':DecisionTreeClassifier(random_state=10,min_samples_leaf=2),
               'RF':RandomForestClassifier(random_state=10,n_estimators=100,n_jobs=-1),
               'GBC':GradientBoostingClassifier(random_state=10,n_estimators=400,learning_rate=0.2),
               'SGD':SGDClassifier(loss="hinge", penalty="l2"),
               'SVM':svm.SVC(kernel='linear', C=0.1),
               'NB':MultinomialNB(alpha=.05)}

In [17]:
#Pipeline to reuse the code
def ML_Pipeline(clf_name):
    clf = Classifiers[clf_name]
    fit = clf.fit(train_features,train['Deal_Status'])
    pred = clf.predict(test_features)
    Accuracy = accuracy_score(test['Deal_Status'],pred)
    Confusion_matrix = confusion_matrix(test['Deal_Status'],pred)
    print('==='*20)
    print('Accuracy = '+str(Accuracy))
    print('==='*20) 
    print(Confusion_matrix)

# # ML Code in Individual Blocks

In [18]:
ML_Pipeline('LR')

Accuracy = 0.5
[[11 52]
 [19 60]]


In [19]:
ML_Pipeline('DTC')

Accuracy = 0.4859154929577465
[[44 19]
 [54 25]]


In [20]:
ML_Pipeline('RF')

Accuracy = 0.5563380281690141
[[ 2 61]
 [ 2 77]]


In [21]:
ML_Pipeline('GBC')

Accuracy = 0.4788732394366197
[[ 7 56]
 [18 61]]


In [22]:
ML_Pipeline('NB')

Accuracy = 0.4084507042253521
[[12 51]
 [33 46]]


In [23]:
ML_Pipeline('SVM')

Accuracy = 0.47183098591549294
[[12 51]
 [24 55]]


In [24]:
ML_Pipeline('SGD')

Accuracy = 0.5140845070422535
[[32 31]
 [38 41]]
